In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [3]:
transform = transforms.Compose([transforms.ToTensor()])

mnist_train = datasets.MNIST(root='data/',download=False, train=True, transform=transform)
mnist_test = datasets.MNIST(root='data/', download=False, train=False, transform=transform)


In [4]:
#Loading dataset into dataloader
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=False)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
     
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2))
        
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.5)

        self.fc1 = nn.Linear(128*7*7, 50,bias=True)
        
        self.fc2 = nn.Linear(50, 10,bias=True)


    def forward(self, x):
        
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)

        out = self.dropout(out)

        
        out = self.fc1(out)
        out = self.sigmoid(out)
        out = self.fc2(out)
        out = self.softmax(out)

        
        return out


In [6]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()


model = CNN()


optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()


In [7]:
training_epochs = 15
total_batch = len(mnist_train) // batch_size

def compute_accuracy(Y_target, hypothesis):
    _, predicted = torch.max(hypothesis.data, 1)
    total = Y_target.size(0)
    correct = (predicted == Y_target).sum().item()

    return correct/total


print("Training Started")

for epoch in range(training_epochs):
    avg_cost = 0
    for i, (X, Y) in enumerate(train_loader):
        print(Y)
        optimizer.zero_grad()

        hypothesis = model(X)
        print
        cost = criterion(hypothesis, Y)

        cost.backward()

        optimizer.step()

        accu = compute_accuracy(Y, hypothesis)
 
        if i % 200 == 0:
            print("Epoch= {},\t batch = {},\t cost = {:.4f},\t accuracy = {}".format(epoch, i, cost.item(), accu))

        avg_cost += cost.data / total_batch

    print("Epoch: {}, averaged cost = {:.4f}".format(epoch + 1, avg_cost.item()))


print('Learning Finished!')

Training Started
tensor([2, 6, 0, 3, 4, 7, 4, 6, 7, 3, 6, 9, 7, 4, 4, 8, 3, 3, 1, 6, 1, 1, 0, 7,
        6, 4, 4, 1, 1, 6, 0, 7])
Epoch= 0,	 batch = 0,	 cost = 2.2959,	 accuracy = 0.15625
tensor([2, 2, 8, 7, 3, 1, 7, 8, 3, 7, 8, 9, 4, 4, 6, 6, 6, 4, 1, 2, 3, 9, 4, 9,
        8, 4, 4, 5, 5, 7, 4, 6])
tensor([4, 1, 0, 9, 2, 2, 5, 5, 7, 1, 0, 5, 5, 3, 2, 3, 7, 3, 1, 8, 2, 4, 5, 4,
        3, 6, 5, 1, 5, 1, 0, 1])
tensor([4, 9, 9, 7, 5, 7, 4, 6, 5, 8, 6, 6, 3, 1, 1, 9, 3, 1, 1, 5, 3, 4, 1, 6,
        1, 4, 2, 9, 3, 9, 9, 2])
tensor([5, 8, 8, 6, 8, 0, 5, 2, 8, 0, 1, 7, 7, 1, 6, 2, 2, 5, 4, 5, 3, 9, 4, 5,
        1, 6, 6, 9, 5, 7, 0, 2])
tensor([1, 2, 6, 7, 1, 5, 5, 7, 4, 7, 8, 5, 1, 5, 1, 9, 6, 7, 7, 1, 8, 6, 3, 6,
        5, 3, 4, 0, 1, 7, 9, 8])
tensor([1, 0, 2, 8, 0, 4, 4, 4, 3, 1, 9, 8, 0, 2, 4, 8, 2, 3, 1, 1, 0, 6, 1, 4,
        7, 6, 9, 6, 0, 4, 4, 3])
tensor([4, 7, 5, 3, 9, 6, 7, 6, 9, 1, 0, 9, 4, 9, 9, 6, 3, 2, 6, 4, 3, 3, 5, 1,
        9, 4, 8, 6, 8, 3, 3, 5])
tensor([3, 9, 1, 1, 4

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images)
        labels = Variable(labels)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
